In [ ]:
%matplotlib inline
from pathlib import Path
from time import monotonic, sleep

import numpy as np
import matplotlib.pyplot as plt
import math

import qcodes as qc
from qcodes.dataset import (
    Measurement,
    initialise_or_create_database_at,
    load_by_guid,
    load_by_run_spec,
    load_or_create_experiment,
    plot_dataset,
)
from qcodes.dataset.descriptions.detect_shapes import detect_shape_of_measurement
from qcodes.logger import start_all_logging
start_all_logging()

from scipy.optimize import curve_fit
import numpy as np

from ultolib import (anritsu, korad, spincore)
from ultolib.spincore import pulse
import qcodes.instrument_drivers.stanford_research as stanford_research

from spinapi import *
from PulseBlasterUSB import *

In [ ]:
pulseblaster = PulseBlasterUSB()
lock_in_amp = stanford_research.SR830(name='lock_in_amp', address='ASRL5::INSTR', terminator='\r')
microwave_src=anritsu.MG3681A(name='microwave_src', address='ASRL4::INSTR', terminator='\r\n')
lock_in_amp.input_config('I 1M')
microwave_src.IQ_modulation('EXT')
pulseblaster.Stop()
microwave_src.output('OFF')
microwave_src.output_level_unit('dBm')

dc_supply = korad.KD3305P('dc_supply', 'ASRL6::INSTR')
dc_supply.ch1.voltage_setpoint(0)
dc_supply.ch1.current_setpoint(0)

## Task 2.3: The Hyperfine Splitting

The negatively charged vacancy in the NV center couples to the spin of its 14N nuclei which is a non-zero spin system. This will lead to further splitting of the electronic energy levels, known as hyperfine splitting.

For this task set the magnetic field to 0.005 T. 

Note: For this task, Task 2.2 must have been completed and the coil_constant determined.

In [ ]:
coil_constant = 
dc_supply.ch1.voltage_setpoint()
B0 = 
I0 = B0/coil_constant
assert I0 <= 0.5, f"Set I0 to be less than 0.5 A. You tried {I0}"
dc_supply.ch1.current_setpoint(I0)
print('The applied current is {:.3f} A.'.format(I0))
dc_supply.ch1.current_setpoint()
#Find the coil constant from the data tomorrow and then continue from there.

Perform an ODMR experiment to find the resonance peak at the highest frequency. Ensure that the coil current is not so high that the resonance of the peak exceeds 3 GHz. If that is the case, lower the B0 in the above code cell. You can consult your ODMR map to get an idea about the frequency range. For this ODMR peak, we will perform a high resolution ODMR scan centered around the chosen peak. Enter the values for the minimum and maximum frequencies to be scanned in the spectrum, below. Initially, it may help to scan a larger range in order to locqate the peak and then tighten the range to get a better scan.

Repeat this experiment for microwave powers 9 dbm, 6 dBm and -3 dB. You may have to increase the lock-in amplifier time constant at -3 dBm.

Note: This experiments uses the pulse sequence from Task 2.1. If you are doing the tasks in order then you can run the code segment below. However, if this task is being done after Task 2.4 or you are unsure whether the pulse sequence is correct re-run the first two code segments of Task 2.1 prior to the following code sequence.

**The microwave frequency must be $\leq 3$ GHz.**

In [ ]:
#Use this to make a parameter out of anything!
MW= qc.ManualParameter('Frequency', unit='Hz')
LI_R = qc.ManualParameter('Signal', unit='V')

#We start by stopping the laser pulsing. This way we can properly initialize.
initialise_or_create_database_at(Path.cwd() / "Hyperfine Splitting.db")
experiment = load_or_create_experiment(
    experiment_name='Hyperfine Splitting',
    sample_name=""
)

meas = Measurement(exp=experiment, name='Hyperfine Splitting')
meas.register_parameter(MW)  # register the first independent parameter
meas.register_parameter(LI_R) # now register the dependent oone

In [ ]:
#Your control code here

In [ ]:
pulseblaster.Stop()
microwave_src.power()             #Repeat for 6 and -3.
microwave_src.output('OFF')

lock_in_amp.time_constant()
lock_in_amp.sensitivity()

#TODO: Enter the ODMR frequency range.
#NOTE: Must not be > 3 GHz (why not? Couldn't we interleave the channels part way through to look at a larger range?)
#TODO: Enter the ODMR frequency range.
min_frequency =                  #Minimum frequency for the spectrum
max_frequency =                   #Maximum frequency for the spectrum, #Must not be > 3 GHz.
num_freq_step = 

In [ ]:
#Run the experiment
microwave_src.output('ON')
with meas.run() as datasaver:
    #Your measurement code here
    ODMR_data = datasaver.dataset  # convenient to have for data access and plotting